upload datafiler


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/MyDrive


 018.gslides
 057.mp4
'1 (1).png'
 1.png
 20170919_102836.dng
 20170919_112137.dng
 20170919_112210.dng
 20170919_134448.dng
 20170919_135358.dng
 20170919_142131.dng
 20170919_142237.dng
 20170919_142606.dng
 20170919_142606.jpg
'20170919_182643 (1).dng'
 20170919_182643.dng
 20170919_185522.dng
 20170922_100434.jpg
 20180109_092615.jpg
 20180109_095236.jpg
 20180109_103058.jpg
 20180117_112624.jpg
 20180117_112814.jpg
 20180118_082549.jpg
 20180123_090855.jpg
 20180123_091323.jpg
 20180125_080800.jpg
 20180125_080812.jpg
 20180301_122922.jpg
 20190415_190327.jpg
 20190705_110653.jpg
 20190705_110710.jpg
 20200224_104901.jpg
 20200224_104930.jpg
 20210206_232519.jpg
 20210530_152203.jpg
 20210630_100013.jpg
 20210817_162146.jpg
 2_02.gdraw
'2 (1).png'
 2.gdraw
 2.png
 3ds
'3d_til plakat'
 3.png
'4 (1).png'
 4.png
'50_transparent (1).png'
 50_transparent.png
'5 (1).png'
 5.png
 Acetylsalicylsyre.gdoc
 activitybehaviour_final.png
 activity_behaviour.png
 activity.drawio
 activity_finalv

In [7]:
!cp "/content/drive/MyDrive/data.zip" /content/


In [8]:
!unzip data.zip -d data/


Streaming output truncated to the last 5000 lines.
  inflating: data/data/eurasian dog/Image_34.jpg  
  inflating: data/data/eurasian dog/Image_25.jpg  
  inflating: data/data/eurasian dog/Image_1.jpg  
  inflating: data/data/eurasian dog/Image_17.jpg  
  inflating: data/data/eurasian dog/Image_28.jpg  
  inflating: data/data/eurasian dog/Image_33.jpg  
  inflating: data/data/eurasian dog/Image_5.jpg  
  inflating: data/data/eurasian dog/Image_10.jpg  
  inflating: data/data/eurasian dog/Image_20.jpg  
  inflating: data/data/eurasian dog/Image_35.jpg  
  inflating: data/data/eurasian dog/Image_22.jpg  
  inflating: data/data/eurasian dog/Image_19.jpg  
  inflating: data/data/eurasian dog/Image_27.jpg  
  inflating: data/data/eurasian dog/Image_11.jpg  
  inflating: data/data/eurasian dog/Image_26.jpg  
  inflating: data/data/eurasian dog/Image_24.jpg  
  inflating: data/data/eurasian dog/Image_9.jpg  
  inflating: data/data/eurasian dog/Image_7.jpeg  
  inflating: data/data/eurasian do

In [10]:
!mv data/data/* data/
!rm -r data/data


In [11]:
import os

print("Top mappe:", os.listdir("data")[:5])
print("Antal mapper (racer):", len(os.listdir("data")))



Top mappe: ['pekingese dog', 'auvergne pointer dog', 'hokkaido dog', 'majorca mastiff dog', 'puli dog']
Antal mapper (racer): 358


In [12]:
!pip install -q tensorflow pillow numpy matplotlib scikit-learn


Opret ImageDataGenerator (forbehandling + data augmentation)


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("ok")
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    "data",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

val_gen = train_datagen.flow_from_directory(
    "data",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)


ok
Found 10074 images belonging to 355 classes.
Found 2487 images belonging to 355 classes.


### Transfer Learning med MobileNetV2

I denne del opbygges en billedklassifikationsmodel ved hjælp af **transfer learning**.  
Vi anvender den fortrænede model **MobileNetV2**, som allerede er trænet på ImageNet-datasættet med millioner af billeder. Det betyder, at modellen allerede kan genkende generelle visuelle mønstre som kanter, former og teksturer.

Den oprindelige klassifikationsdel fjernes (`include_top=False`), så kun den konvolutionelle feature-extractor bevares. Disse lag fryses (`trainable=False`), så deres vægte ikke ændres under træningen. På den måde genbruges den indlærte billedforståelse.

Ovenpå denne base-model tilføjes en ny klassifikationsdel, som er tilpasset vores datasæt:

- **GlobalAveragePooling2D** samler features fra CNN’en
- **Dense lag (256 neuroner)** lærer mønstre specifikke for vores klasser
- **Dropout** reducerer overfitting
- **Output-lag (Softmax)** giver sandsynligheder for hver klasse

Denne metode gør det muligt at opnå høj præcision med mindre data og kortere træningstid.


In [15]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model

NUM_CLASSES = 355  # eller len(class_names)

base_model = MobileNetV2(
    weights="imagenet",  # ← vigtigt
    include_top=False,
    input_shape=(224,224,3)
)
base_model.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu", name="dense_1")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="dense_2")(x)

model = Model(inputs=base_model.input, outputs=outputs)


### Kompilering af modellen

Før modellen kan trænes, skal den **kompileres**. Dette trin definerer, hvordan modellen lærer.

- **Optimizer (`adam`)** styrer hvordan vægtene justeres under træning. Adam er en effektiv og ofte anvendt metode til deep learning.
- **Loss function (`categorical_crossentropy`)** måler hvor stor forskel der er mellem modellens forudsigelser og de rigtige labels. Denne bruges ved flerklasses klassifikation.
- **Metrics (`accuracy`)** bruges til at evaluere modellens præstation under træning og validering.

Kompileringen fastlægger altså de matematiske regler for læringsprocessen.


In [20]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]   # ← liste!
)


### Gemning af den trænede model

Efter træningen gemmes modellen, så den kan genbruges uden at skulle trænes igen.

Filen indeholder:
- Modelarkitektur  
- Trænede vægte  
- Kompileringsindstillinger  

Dette gør det muligt senere at indlæse modellen og bruge den til at lave forudsigelser på nye billeder.

Det er vigtigt, at modellen gemmes **efter træning**, så den lagrer den indlærte viden.


In [21]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stop]
)


Epoch 1/20
  2/315 ━━━━━━━━━━━━━━━━━━━━ 13:04 3s/step - accuracy: 0.0000e+00 - loss: 6.2298

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


315/315 ━━━━━━━━━━━━━━━━━━━━ 832s 3s/step - accuracy: 0.0553 - loss: 5.3685 - val_accuracy: 0.3172 - val_loss: 2.8901
Epoch 2/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 818s 3s/step - accuracy: 0.2554 - loss: 3.1188 - val_accuracy: 0.3868 - val_loss: 2.3945
Epoch 3/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 827s 3s/step - accuracy: 0.3391 - loss: 2.5866 - val_accuracy: 0.4234 - val_loss: 2.2135
Epoch 4/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 863s 3s/step - accuracy: 0.3961 - loss: 2.3075 - val_accuracy: 0.4371 - val_loss: 2.1317
Epoch 5/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 782s 2s/step - accuracy: 0.4190 - loss: 2.1707 - val_accuracy: 0.4612 - val_loss: 2.0747
Epoch 6/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 791s 3s/step - accuracy: 0.4517 - loss: 2.0153 - val_accuracy: 0.4644 - val_loss: 2.0321
Epoch 7/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 787s 2s/step - accuracy: 0.4763 - loss: 1.9162 - val_accuracy: 0.4664 - val_loss: 2.0177
Epoch 8/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 804s 3s/step - accuracy: 0.4954 - loss: 1.8047 - val_accuracy: 0.465

In [23]:
model.save("dog_breed_model.h5")

### Gemning af klasse-navne

For at kunne tolke modellens forudsigelser senere gemmes en liste over klasse-navne i en separat fil.

`flow_from_directory()` opretter automatisk en mapping mellem mappenavne (klasser) og numeriske labels. Modellen arbejder kun med disse tal, men for at kunne oversætte en forudsigelse tilbage til et menneskeligt læsbart navn, skal denne mapping gemmes.

Først vendes dictionary’en `class_indices`, så vi får en mapping fra indeks → klassenavn. Derefter oprettes en liste i korrekt rækkefølge, så indeks i modellens output svarer direkte til positionen i listen.

Til sidst gemmes listen i en JSON-fil, så den kan indlæses senere ved inference, når modellen bruges på nye billeder.


In [23]:
import json

idx_to_class = {v: k for k, v in train_gen.class_indices.items()}
class_names = [idx_to_class[i] for i in range(len(idx_to_class))]

with open("class_names.json", "w") as f:
    json.dump(class_names, f)


In [25]:
from google.colab import files
files.download("dog_breed_model.h5")
files.download("class_names.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
from tensorflow.keras.models import load_model

old_model = load_model("dog_model.h5", compile=False)
model.set_weights(old_model.get_weights())


In [20]:
model.get_layer("dense_1").set_weights(old.get_layer("dense_2").get_weights())
model.get_layer("dense_2").set_weights(old.get_layer("dense_3").get_weights())


In [10]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(355, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)


In [14]:
model.save_weights("dog.weights.h5")


In [ ]:
from google.colab import files
files.download("dog.weights.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>